In [1]:
import sqlite3
import time
import numpy as np
import pandas as pd

from sqlite3 import Error
from pprint import pprint
from flask import Flask, jsonify

# https://medium.com/alpha-vantage/get-started-with-alpha-vantage-data-619a70c7f33a
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators

# Import API key
from api_keys import api_key

# Technically, these are all constants
old_to_new = {"1. open": "open", "2. high": "high", "3. low": "low", "4. close": "close", "5. volume": "volume"}
reorder = ["date", "ticker", "open", "high", "low", "close", "volume"]

# Create a list of ticker symbols
# stocks = ["DAL", "AAL", "LUV", "UAL", "MSFT", "AAPL", "V", "INTC", "NEE", "D", "DUK", "SO", "FB", "GOOG", "GOOGL", "TMUS", "UBER", "BKNG", "LYFT", "TCOM", "ZM"]
stocks = ["JETS", "XLK", "VPU", "XLC", "AWAY"]

### SETUP ONLY

### CREATE OUR CATEGORIES

In [12]:
try:
    # Store our stock data in a table
    # first, create our dataframe of information
#     stockInfo = {"category": ["JETS", "JETS", "JETS", "JETS", "XLK", "XLK", "XLK", "XLK", "VPU", "VPU", "VPU", "VPU", "XLC", "XLC", "XLC", "XLC", "AWAY", "AWAY", "AWAY", "AWAY", "MISC"],
#                  "ticker": ["DAL", "AAL", "LUV", "UAL", "MSFT", "AAPL", "V", "INTC", "NEE", "D", "DUK", "SO", "FB", "GOOG", "GOOGL", "TMUS", "UBER", "BKNG", "LYFT", "TCOM", "ZM"],
#                  "company": ["Delta Air Lines, Inc.", "American Airlines Group Inc.", "Southwest Airlines Co.", "United Airlines Holdings, Inc.",
#                             "Microsoft Corporation", "Apple Inc.", "Visa Inc.", "Intel Corporation",
#                             "NextEra Energy, Inc.", "Dominion Energy, Inc.", "Duke Energy Corporation", "The Southern Company",
#                             "Facebook, Inc.", "Alphabet Inc. (Class C)", "Alphabet Inc. (Class A)", "T-Mobile US, Inc.",
#                             "Uber Technologies, Inc.", "Booking Holdings Inc.", "Lyft, Inc.", "Trip.com Group Limited", 
#                             "Zoom Video Communications, Inc."]}
    # ETFs Only
    stockInfo = {"category": ["JETS", "XLK", "VPU", "XLC", "AWAY"], 
                 "ticker": ["JETS", "XLK", "VPU", "XLC", "AWAY"],
                 "company": ["JETS invests in both U.S. and non-U.S. companies involved with the airline industry, including passenger airlines, aircraft manufacturers, airports and terminal services companies.", 
                             "XLK tracks an index of S&P 500 technology stocks.", 
                             "VPU tracks a broad-based, market-cap-weighted index of US utilities companies.", 
                             "XLC tracks a market-cap-weighted index of US telecommunication and media & entertainment components of the S&P 500 index.", 
                             "AWAY tracks an index of global travel technology companies that are fundamentally screened and are weighted based on market cap and liquidity."
                            ]
    }
    
    # Create the pandas DataFrame 
    df_categories = pd.DataFrame(stockInfo, columns=["category", "ticker", "company"]) 

    df_categories.set_index("category", inplace=True)
    
    # get our connection
    conn = sqlite3.connect(r"static/data/stocks.sqlite")

    # create our cursor
    cur = conn.cursor()

    # first check to see if our table exists
#     cur.execute("DROP TABLE IF EXISTS categories;")

    # add this stock data to our database
    df_categories.to_sql(name="categories", con=conn, if_exists="append", index=True)

except Error as e:
        print(f"Save failed for the categories table. Please try again later./n/n  Error is:  {e}")
finally:
    # make sure we close our connection
    conn.close()
# end try

In [13]:
# # TEST - pull data from our database to make sure it's all there and type casted correctly
# # get our connection
conn = sqlite3.connect(r"static/data/stocks.sqlite")

cur = conn.cursor()
cur.execute("select * from categories")
myDF = pd.DataFrame(cur.fetchall(), columns=["category", "ticker", "company"])
conn.close()

print(myDF)

   category ticker                                            company
0      JETS    DAL                              Delta Air Lines, Inc.
1      JETS    AAL                       American Airlines Group Inc.
2      JETS    LUV                             Southwest Airlines Co.
3      JETS    UAL                     United Airlines Holdings, Inc.
4       XLK   MSFT                              Microsoft Corporation
5       XLK   AAPL                                         Apple Inc.
6       XLK      V                                          Visa Inc.
7       XLK   INTC                                  Intel Corporation
8       VPU    NEE                               NextEra Energy, Inc.
9       VPU      D                              Dominion Energy, Inc.
10      VPU    DUK                            Duke Energy Corporation
11      VPU     SO                               The Southern Company
12      XLC     FB                                     Facebook, Inc.
13      XLC   GOOG  

### SETUP ONLY

### Create our covid19 table with news information

In [ ]:
try:
    # Store our stock data in a table
    # first, create our dataframe of information
    # https://www.cnn.com/2020/02/06/health/wuhan-coronavirus-timeline-fast-facts/index.html
    stockInfo = {"date": ["2019-12-31", "2020-01-07", "2020-01-13", "2020-01-16", "2020-01-21", "2020-01-23", "2020-01-29", "2020-01-30", "2020-01-31", "2020-02-07", "2020-02-11", "2020-02-12", "2020-02-26", "2020-03-02", "2020-03-03", "2020-03-04", "2020-03-11", "2020-03-13", "2020-03-18", "2020-03-27", "2020-04-02"],
                 "description": ["2019-12-31 - Cases of pneumonia detected in Wuhan, China, are first reported to the WHO. During this reported period, the virus is unknown. The cases occur between December 12 and December 29, according to Wuhan Municipal Health.", 
                                "2020-01-07 - Chinese authorities confirm that they have identified the virus as a novel coronavirus, initially named '2019-nCoV' by the WHO.", 
                                "2020-01-11 (Saturday) - The Wuhan Municipal Health Commission announces the first death caused by the coronavirus. A 61-year-old man, exposed to the virus at the seafood market, died on January 9 after respiratory failure caused by severe pneumonia.", 
                                "2020-01-16 - Japanese authorities confirm that a Japanese man who traveled to Wuhan is infected with the virus.", 
                                "2020-01-21 - Officials in Washington state confirm the first case on US soil.", 
                                "2020-01-23 - At an emergency committee convened by the World Health Organization, the WHO says that the Wuhan coronavirus does not yet constitute a public health emergency of international concern. The Beijing Culture and Tourism Bureau cancels all large-scale Lunar New Year celebrations in an effort to contain the growing spread of Wuhan coronavirus. On the same day, Chinese authorities enforce a partial lockdown of transport in and out of Wuhan. Authorities in the nearby cities of Huanggang and Ezhou Huanggang announce a series of similar measures.", 
                                "2020-01-29 - The White House announces the formation of a new task force that will help monitor and contain the spread of the virus, and ensure Americans have accurate and up-to-date health and travel information, it says.", 
                                "2020-01-30 - The United States reports its first confirmed case of person-to-person transmission of the Wuhan coronavirus. On the same day, the WHO determines that the outbreak constitutes a Public Health Emergency of International Concern (PHEIC).", 
                                "2020-01-31 - The Donald Trump administration announces it will deny entry to foreign nationals who have traveled in China in the last 14 days.", 
                                "2020-02-07 - Li Wenliang, a Wuhan doctor who was targeted by police for trying to sound the alarm on a 'SARS-like' virus in December, dies of the coronavirus. Following news of Li's death, the topics 'Wuhan government owes Dr. Li Wenliang an apology,' and 'We want freedom of speech,' trend on China's Twitter-like platform, Weibo, before disappearing from the heavily censored platform.", 
                                "2020-02-11 - The WHO names the coronavirus 'Covid-19'.", 
                                "2020-02-12 - The CDC changes criteria for counting confirmed cases of novel coronavirus in the United States and begins tracking two separate and distinct groups: those repatriated by the US Department of State and those identified by the US public health network.", 
                                "2020-02-26 - CDC officials say that a California patient being treated for novel coronavirus is the first US case of unknown origin. The patient, who didn't have any relevant travel history nor exposure to another known patient, is the first possible US case of 'community spread.'", 
                                "2020-02-29 (Saturday) - A state health official announces that a patient infected with the novel coronavirus in Washington state has died, marking the first death due to the virus in the United States. Washington Gov. Jay Inslee declares a state of emergency, directing state agencies to use all resources necessary to respond to the outbreak. 2020-03-01 (Sunday) - Florida Gov. Ron DeSantis declares a public health emergency in the state of Florida. Over the next several days, Kentucky, New York, Maryland, Utah and Oregon declare states of emergency.", 
                                "2020-03-03 - The Federal Reserve slashes interest rates by half a percentage point in an attempt to give the US economy a jolt in the face of concerns about the coronavirus outbreak. It is the first unscheduled, emergency rate cut since 2008, and it also marks the biggest one-time cut since then.", 
                                "2020-03-04 - The CDC formally removes earlier restrictions that limited coronavirus testing of the general public to people in the hospital, unless they had close contact with confirmed coronavirus cases. According to the CDC, clinicians should now 'use their judgment to determine if a patient has signs and symptoms compatible with COVID-19 and whether the patient should be tested.'", 
                                "2020-03-11 - The WHO declares the novel coronavirus outbreak to be a pandemic. WHO says the outbreak is the first pandemic caused by a coronavirus. In an Oval Office address, Trump announces that he is restricting travel from Europe to the United States for 30 days in an attempt to slow the spread of coronavirus. The ban, which applies to the 26 countries in the Schengen Area, applies only to foreign nationals and not American citizens and permanent residents who'd be screened before entering the country.", 
                                "2020-03-13 - Trump declares a national emergency to free up $50 billion in federal resources to combat coronavirus. Colleges, High Schools, Middle Schools and Elementary Schools begin closing around the country.", 
                                "2020-03-18 - Trump signs into law a coronavirus relief package that includes provisions for free testing for Covid-19 and paid emergency leave.", 
                                "2020-03-27 - Trump signs the $2 trillion stimulus package into law to offset the economic damage of coronavirus, producing one of the most expensive and far-reaching measures in the history of Congress.", 
                                "2020-04-02 - According to the Department of Labor, 6.6 million US workers file for their first week of unemployment benefits in the week ending March 28, the highest number of initial claims in history. Globally, the total number of coronavirus cases surpasses 1 million, according to Johns Hopkins University's tally."]}
    
    # Create the pandas DataFrame 
    df_categories = pd.DataFrame(stockInfo, columns=["date", "description"]) 

    df_categories.set_index("date", inplace=True)
    
    # get our connection
    conn = sqlite3.connect(r"static/data/stocks.sqlite")

    # create our cursor
    cur = conn.cursor()

    # first check to see if our table exists
    if (delTable("covid19") == True):
        # add this stock data to our database
        df_categories.to_sql(name="covid19", con=conn, if_exists="append", index=True)
    # end if
    
except Error as e:
        print(f"Save failed for the covid-19 table. Please try again later./n/n  Error is:  {e}")
finally:
    # make sure we close our connection
    conn.close()
# end try

In [ ]:
# # TEST - pull data from our database to make sure it's all there and type casted correctly
# # get our connection
conn = sqlite3.connect(r"static/data/stocks.sqlite")

cur = conn.cursor()
cur.execute("select * from covid19")
myDF = pd.DataFrame(cur.fetchall(), columns=["date", "description"])
conn.close()

print(myDF)

###  Start production data management

In [2]:
# Function to get category from ticker
# returns string
def getCategory(ticker):
    mygetCategory = ""
    
    try:
        conn = sqlite3.connect(r"static/data/stocks.sqlite")

        cur = conn.cursor()
        cur.execute(f"SELECT category FROM categories WHERE ticker = '{ticker}'")
        rows = cur.fetchall()
        conn.close()
        
        for row in rows:
            mygetCategory = row[0]
        # next row
        
    except Error as e:
        print(f"The API failed for the stock '{ticker}'. Please try again later./n/n  Error is:  {e}")
    finally:
        conn.close()
    # end try
    
    return mygetCategory
# end getCategory(ticker)

In [3]:
# Function to get tickers from category
# returns tuple
def getStocks(category):
    mygetStocks = []
    
    try:
        conn = sqlite3.connect(r"static/data/stocks.sqlite")

        cur = conn.cursor()
        cur.execute(f"SELECT ticker FROM categories WHERE category = '{category}'")
        rows = cur.fetchall()
        conn.close()
        for row in rows:
            mygetStocks.append(row[0])
        # next row
        
    except Error as e:
        print(f"The API failed for the category '{category}'. Please try again later./n/n  Error is:  {e}")
    finally:
        conn.close()
    # end try
    
    return mygetStocks
# end getStocks()

In [4]:
# Test the function above
print(getStocks("JETS"))

['DAL', 'AAL', 'LUV', 'UAL']


In [5]:
print(getCategory("MSFT"))

XLK


In [2]:
def storeData(df_Stock):
    try:
        # store the ticker sykmbol
        storeDataticker = df_Stock["ticker"][0]
        
        # get our connection
        conn = sqlite3.connect(r"static/data/stocks.sqlite")
        
        # create our cursor
        cur = conn.cursor()
        
        # first check to see if our table exists
        cur.execute("SELECT count(name) FROM sqlite_master WHERE type='table' AND name='stocks'")

        # if the count is 1, then our table exists
        if cur.fetchone()[0] == 1:
            cur.execute(f"DELETE FROM stocks WHERE ticker = '{storeDataticker}'")
        # end if
        print(df_Stock)
        # add this stock data to our database
        df_Stock.to_sql(name="stocks", con=conn, if_exists="append", index=True)
    
    except Error as e:
            print(f"Save failed for the stock: '{storeDataticker}'. Please try again later./n/n  Error is:  {e}")
    finally:
        # make sure we close our connection
        conn.close()
    # end try
# end storeData()

In [4]:
def delTable(tableName):
    retVal = False
    
    try:
        # get our connection
        conn = sqlite3.connect(r"static/data/stocks.sqlite")
        
        # create our cursor
        cur = conn.cursor()
        cur.execute(f"DROP TABLE IF EXISTS {tableName}")
        retVal = True
    except Error as e:
            print(f"Could not drop the table named '{tableName}'./n/n  Error is:  {e}")
            retVal = False
    finally:
        # make sure we close our connection
        conn.close()
        return retVal
    # end try
# end delTable(tableName)

In [3]:
def refresh():
    # Delete the stocks table.  We'll add it back as we add data
#     if (delTable("stocks") == True):
    for stockTicker in stocks:
        print(f"Getting data for ticker:'{stockTicker}'")

        try:
            # Chose your output format, or default to JSON (python dict)
            ts = TimeSeries(api_key, output_format='pandas')

            # aapl_data is a pandas dataframe, aapl_meta_data is a dict
            stockData, meta_data = ts.get_daily(symbol=stockTicker)

            # add the ticker column
            stockData["ticker"] = stockTicker

            # because "date" is not a field but rather an index
            # reset the index so "date" is a field
            stockData.reset_index(inplace=True)

            # rename the columns
            stockData.rename(columns=old_to_new, inplace=True)

            # reorder the columns
            stockData = stockData[reorder]  

            # convert to datetime
            stockData["date"] = pd.to_datetime(stockData["date"])

            # set the index
            stockData.set_index("date", inplace=True)

            # convert the numeric fields to float
            stockData[["open", "high", "low", "close", "volume"]] = stockData[["open", "high", "low", "close", "volume"]].apply(pd.to_numeric)

            # stick all the data into our SQLite database
            storeData(stockData)

            time.sleep(12)
        except Error as e:
            print(f"The API failed for the stock '{stockTicker}'. Please try again later./n/n  Error is:  {e}")
        finally:
            print("The update process has completed.")
            # End Try
        # Next stockTicker
    # end if
# end refresh()

In [5]:
# Test the refresh function
refresh()

Getting data for ticker:'JETS'
           ticker   open     high      low    close     volume
date                                                          
2020-04-24   JETS  13.55  13.6000  13.0600  13.3600  1430518.0
2020-04-23   JETS  13.54  13.7850  13.4000  13.4000  1644048.0
2020-04-22   JETS  13.95  13.9900  13.1300  13.4000  2610378.0
2020-04-21   JETS  13.75  13.9800  13.5500  13.7300  2182115.0
2020-04-20   JETS  14.07  14.2200  13.6500  13.9900  2184369.0
...           ...    ...      ...      ...      ...        ...
2019-12-06   JETS  31.69  31.6900  31.5650  31.5650     2317.0
2019-12-05   JETS  31.38  31.4100  31.2464  31.2525     4267.0
2019-12-04   JETS  31.33  31.6046  31.3300  31.3600     4534.0
2019-12-03   JETS  31.41  31.4100  30.8200  31.0925    16554.0
2019-12-02   JETS  31.92  32.0700  31.5600  31.5622     7574.0

[100 rows x 6 columns]
The update process has completed.
Getting data for ticker:'XLK'
           ticker   open     high      low  close      volume


In [8]:
# TEST - pull data from our database to make sure it's all there and type casted correctly
# get our connection
conn = sqlite3.connect(r"static/data/stocks.sqlite")

cur = conn.cursor()
cur.execute("select * from stocks")
myDF = pd.DataFrame(cur.fetchall(), columns=["date", "ticker", "open", "high", "low", "close", "volume"])
conn.close()
myDF

,date,ticker,open,high,low,close,volume
0,2020-04-17 00:00:00,DAL,24.1500,24.61,23.59,24.27,52488030.0
1,2020-04-16 00:00:00,DAL,23.7600,23.80,22.63,22.78,55973221.0
2,2020-04-15 00:00:00,DAL,24.9100,25.50,23.31,24.35,87798348.0
3,2020-04-14 00:00:00,DAL,23.9899,25.29,23.83,24.54,58599118.0
4,2020-04-13 00:00:00,DAL,24.9800,25.06,22.08,23.25,75904867.0
...,...,...,...,...,...,...,...
2095,2019-11-29 00:00:00,ZM,74.1000,74.94,73.86,74.50,512643.0
2096,2019-11-27 00:00:00,ZM,75.8100,75.84,73.36,74.08,1356274.0
2097,2019-11-26 00:00:00,ZM,75.0000,75.77,74.76,75.16,1848973.0
2098,2019-11-25 00:00:00,ZM,73.8000,75.06,72.86,74.55,1943187.0


In [8]:
def getStockDataByTicker(ticker):
    mygetStockDataByTicker = []

    try:
        conn = sqlite3.connect(r"static/data/stocks.sqlite")

        cur = conn.cursor()
        cur.execute(f"SELECT date(stocks.date) as date, stocks.ticker, company, open, high, low, close, volume FROM stocks LEFT JOIN categories ON categories.ticker = stocks.ticker WHERE stocks.ticker = '{ticker}'")
        rows = cur.fetchall()

        conn.close()
        for row in rows:
            mygetStockDataByTicker.append({
                "date": row[0],
                "ticker": row[1],
                "company": row[2],
                "open": row[3],
                "high": row[4], 
                "low": row[5],
                "close": row[6],
                "volume": row[7]
            })
#         next row

    except Error as e:
        print(f"The database query failed for the stock '{ticker}'./n/n  Error is:  {e}")
    finally:
        conn.close()
    # end try

    return mygetStockDataByTicker
# end getStockDataByTicker(ticker)

In [9]:
# test
myStocksByTickerData = getStockDataByTicker("AAPL")
myStocksByTickerData

[]

In [11]:
def getStockDataByCategory(category):
    myStocks = []
    
    try:
        conn = sqlite3.connect(r"static/data/stocks.sqlite")

        cur = conn.cursor()
        cur.execute(f"SELECT date(stocks.date), stocks.ticker, categories.company, stocks.open, stocks.high, stocks.low, stocks.close, stocks.volume, covid19.description FROM stocks INNER JOIN categories ON categories.ticker = stocks.ticker INNER JOIN covid19 ON date(covid19.date) = date(stocks.date) WHERE categories.category = '{category}'")
        rows = cur.fetchall()
        conn.close()
        
        for row in rows:
            myStocks.append({
                "date": row[0],
                "ticker": row[1],
                "company": row[2],
                "open": row[3],
                "high": row[4], 
                "low": row[5],
                "close": row[6],
                "volume": row[7],
                "description": row[8]
            })
#         next row
        
    except Error as e:
        print(f"The database query failed for the category '{category}'./n/n  Error is:  {e}")
    finally:
        conn.close()
    # end try
    
    return myStocks
# end getStockDataByCategory(category)

In [12]:
# test
myStocksByCategoryData = getStockDataByCategory("JETS")
myStocksByCategoryData

[{'date': '2019-12-31',
  'ticker': 'DAL',
  'company': 'Delta Air Lines, Inc.',
  'open': 58.49,
  'high': 58.7313,
  'low': 58.34,
  'close': 58.48,
  'volume': 2917296.0,
  'description': '2019-12-31 - Cases of pneumonia detected in Wuhan, China, are first reported to the WHO. During this reported period, the virus is unknown. The cases occur between December 12 and December 29, according to Wuhan Municipal Health.'},
 {'date': '2020-01-07',
  'ticker': 'DAL',
  'company': 'Delta Air Lines, Inc.',
  'open': 57.91,
  'high': 58.07,
  'low': 57.47,
  'close': 57.61,
  'volume': 5613720.0,
  'description': "2020-01-07 - Chinese authorities confirm that they have identified the virus as a novel coronavirus, initially named '2019-nCoV' by the WHO."},
 {'date': '2020-01-13',
  'ticker': 'DAL',
  'company': 'Delta Air Lines, Inc.',
  'open': 59.03,
  'high': 59.7,
  'low': 59.03,
  'close': 59.49,
  'volume': 8402812.0,
  'description': '2020-01-11 (Saturday) - The Wuhan Municipal Health C

In [11]:
conn = sqlite3.connect(r"static/data/stocks.sqlite")

cur = conn.cursor()
cur.execute("SELECT date(stocks.date) as date, stocks.ticker, company, open, high, low, close, volume FROM stocks LEFT JOIN categories ON categories.ticker = stocks.ticker")
rows = cur.fetchall()

conn.close()

print(rows)

[('2020-04-24', 'JETS', None, 13.55, 13.6, 13.06, 13.36, 1430518.0), ('2020-04-23', 'JETS', None, 13.54, 13.785, 13.4, 13.4, 1644048.0), ('2020-04-22', 'JETS', None, 13.95, 13.99, 13.13, 13.4, 2610378.0), ('2020-04-21', 'JETS', None, 13.75, 13.98, 13.55, 13.73, 2182115.0), ('2020-04-20', 'JETS', None, 14.07, 14.22, 13.65, 13.99, 2184369.0), ('2020-04-17', 'JETS', None, 14.76, 14.76, 14.25, 14.35, 3134703.0), ('2020-04-16', 'JETS', None, 14.56, 14.56, 13.73, 13.81, 2921031.0), ('2020-04-15', 'JETS', None, 15.32, 15.34, 14.37, 14.79, 4194084.0), ('2020-04-14', 'JETS', None, 15.15, 15.5, 14.95, 15.01, 3198935.0), ('2020-04-13', 'JETS', None, 15.82, 15.8397, 14.16, 14.56, 2936943.0), ('2020-04-09', 'JETS', None, 15.41, 15.91, 14.72, 15.3, 5682541.0), ('2020-04-08', 'JETS', None, 14.07, 14.36, 13.75, 14.26, 2775612.0), ('2020-04-07', 'JETS', None, 14.29, 14.88, 13.28, 13.41, 3954339.0), ('2020-04-06', 'JETS', None, 12.85, 13.1, 12.359, 12.81, 3104404.0), ('2020-04-03', 'JETS', None, 12.96, 